In [3]:
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bqexercise.json"
client = bigquery.Client()

In [6]:
# Constructing a reference to the 'SEC' dataset

df_ref = client.dataset('sec_quarterly_financials', project = 'bigquery-public-data')

In [7]:
# Building API request

dataset = client.get_dataset(df_ref)

In [8]:
# List tables in 'hacker news' dataset

tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] 

In [9]:
print(list_of_tables)

['calculation', 'dimension', 'measure_tag', 'numbers', 'presentation', 'quick_summary', 'rendering', 'sic_codes', 'submission', 'txt']


In [16]:
# Building reference to the quick_summary table: 
table_ref = df_ref.table('quick_summary')

# API request: fetch the table.
table = client.get_table(table_ref)

In [17]:
table.schema

[SchemaField('submission_number', 'STRING', 'REQUIRED', 'Accession Number. The 20-character string formed from the 18-digit number assigned by the Commission to each EDGAR submission. submission_number: [adsh]', (), ()),
 SchemaField('company_name', 'STRING', 'NULLABLE', 'Name of registrant from the submission table. This corresponds to the name of the legal entity as recorded in EDGAR as of the filing date.', (), ()),
 SchemaField('measure_tag', 'STRING', 'REQUIRED', 'The unique identifier (name) for a tag in a specific taxonomy release. measure_tag: [tag]', (), ()),
 SchemaField('period_end_date', 'STRING', 'REQUIRED', 'The end date for the data value, rounded to the nearest month end. period_end_date: [ddate]', (), ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'The value. This is not scaled, it is as found in the Interactive Data file, but is rounded to four digits to the right of the decimal point.', (), ()),
 SchemaField('units', 'STRING', 'REQUIRED', 'The unit of measure for th

In [18]:
# Previewing the first five lines of the "quick_summary" table:

client.list_rows(table, max_results = 5).to_dataframe()

,submission_number,company_name,measure_tag,period_end_date,value,units,number_of_quarters,version,central_index_key,ein,sic,fiscal_year_end,form,fiscal_year,fiscal_period_focus,date_filed,date_accepted
0,0001437749-17-000102,NATIONAL HOLDINGS CORP,Liabilities,20160930,34773000.0,USD,0,us-gaap/2016,1023844,364128138,6200,0930,S-1/A,2016,FY,20170104,2017-01-04 12:20:00+00:00
1,0001062993-17-000442,"XIANGTIAN (USA) AIR POWER CO., LTD.",CostOfGoodsSold,20140731,0.0,USD,4,us-gaap/2015,1472468,980632932,3621,0731,POS AM,2017,Q1,20170131,2017-01-30 17:34:00+00:00
2,0001564590-17-000589,SCHLUMBERGER LIMITED/NV,DeferredTaxLiabilitiesPropertyPlantAndEquipment,20161231,79000000.0,USD,0,us-gaap/2016,87347,520684746,1389,1231,10-K,2016,FY,20170125,2017-01-25 11:34:00+00:00
3,0001558370-17-000064,HELEN OF TROY LTD,OtherComprehensiveIncomeLossNetOfTaxPortionAtt...,20151130,1805000.0,USD,3,us-gaap/2016,916789,742692550,3634,0229,10-Q,2016,Q3,20170109,2017-01-09 16:00:00+00:00
4,0001674862-17-000006,ASHLAND GLOBAL HOLDINGS INC,DepreciationDepletionAndAmortization,20161231,77000000.0,USD,1,us-gaap/2015,1674862,812587835,5160,0930,10-Q,2017,Q1,20170127,2017-01-27 16:04:00+00:00


In [19]:
# Building reference to the calculation table: 
table_ref_calc = df_ref.table('calculation')

# API request: fetch the table.
table_calc = client.get_table(table_ref_calc)

In [20]:
# Previewing the first five lines of the "calculation" table:

client.list_rows(table_calc, max_results = 5).to_dataframe()

,submission_number,group,arc,negative,parent_tag,parent_version,child_tag,child_version
0,0000277948-09-000066,1,18,1,Assets,us-gaap/2008,Assets,us-gaap/2008
1,0000277948-09-000066,1,29,1,Assets,us-gaap/2008,Assets,us-gaap/2008
2,0000277948-09-000066,1,30,1,Assets,us-gaap/2008,Assets,us-gaap/2008
3,0000277948-09-000066,1,25,1,Assets,us-gaap/2008,Assets,us-gaap/2008
4,0000277948-09-000066,1,28,1,Assets,us-gaap/2008,Assets,us-gaap/2008


In [62]:
join_query = """
            select qs.company_name, qs.ein, sum(value) as total_value
            from `bigquery-public-data.sec_quarterly_financials.quick_summary` as qs
            inner join `bigquery-public-data.sec_quarterly_financials.calculation` as c
                on c.submission_number = qs.submission_number
            where fiscal_year = 2017 and value < 79000000.0
            group by qs.company_name, qs.ein
            order by total_value DESC
            limit 1000
            """

In [63]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(join_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
join_sec = query_job.to_dataframe()

In [64]:
join_sec

,company_name,ein,total_value
0,"ILG, INC.",000000000,8.824925e+12
1,BROOKLINE BANCORP INC,000000000,7.764603e+12
2,BERKSHIRE HILLS BANCORP INC,043510455,7.724081e+12
3,INDEPENDENT BANK CORP /MI/,382032782,7.705029e+12
4,OLD NATIONAL BANCORP /IN/,351539838,7.689947e+12
...,...,...,...
995,MAGNACHIP SEMICONDUCTOR CORP,830406195,1.404848e+12
996,AMBARELLA INC,980459628,1.404671e+12
997,"FORTERRA, INC.",371830464,1.404384e+12
998,NATIONAL HEALTH INVESTORS INC,621470956,1.402561e+12
